In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/ai-residency-batch-3/test.csv',sep=';')
df.head(10)

Brand may affect massively on price

In [ ]:
df['brand']=df['name'].apply( lambda x: x.split(' ')[0])
df

# NaN handle

In [ ]:
df.info(show_counts=True)
# mileage, engine, max_power, torque,seats null attributes

In [ ]:
df[df['name']=='Maruti Swift 1.3 LXI']

In [ ]:
df[df['name']=='Toyota Etios Liva G']

In [ ]:
df.fillna('14.3 kmpl',inplace=True) # few -> drop

In [ ]:
df=df.reset_index().drop('index',axis=1)
df


In [ ]:
df.info(show_counts=True)

processing mileage, engine, max_power,torque columns ( to 1 type)

In [ ]:
df['mileage'].apply(lambda x: x.split(' ')[1]).value_counts()

In [ ]:
df['mileage'][0][:-6]

In [ ]:
def kmkgTokmpl(x):
    if str(x).endswith('km/kg'):
        x = x[:-6]
        x = float(x)*1.40
    elif str(x).endswith('kmpl'):
        x = x[:-4]
    return x
df['mileage']=df['mileage'].apply(kmkgTokmpl)

In [ ]:
type(df['mileage'][0])

In [ ]:
df['mileage'].astype('float16')

In [ ]:
df['engine'].apply(lambda x: x.split(' ')[1]).value_counts()

In [ ]:
df['engine']=df['engine'].apply(lambda x: x.split(' ')[0])

In [ ]:
df['max_power']=df['max_power'].apply(lambda x: x.split(' ')[0])

In [ ]:
df['mileage']=df['mileage'].astype('float64')
df['engine']=df['engine'].astype('float64')
df['max_power']=df['max_power'].astype('float64')

In [ ]:
df['torque'].apply(lambda x: x.split('@')[0][-5:] if (x.find("Nm") == -1) else None).unique()

In [ ]:
df['torque'].apply(lambda x: x.split('@')[0][-3:]).unique()

In [ ]:
df['torque'][30:50] # :)) -> drop

In [ ]:
df.drop('torque',axis=1,inplace=True)

In [ ]:
df['seats'].value_counts() # 2 seats may be luxury car  -> is2seat

In [ ]:
df['is2Seat']=df['seats'].apply(lambda x: 1 if x==2. else 0)
df

In [ ]:
df['is2Seat'].value_counts()

# Category data handle

In [ ]:
df

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for cols in [4,5,6,7,12]:
    col=df.columns[cols]
    df[col]=le.fit_transform(df[col])
df

# Train_test prepare

In [ ]:
df=df.drop('name',axis=1)
df

In [ ]:
df['is2Seat']

In [ ]:
df

In [ ]:
y=df['selling_price']
x=df.drop('selling_price',axis=1)
(x.shape,y.shape)

In [ ]:
x=df

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(x, y, test_size=0.10, random_state=20)
print(X_train.shape)
print(y_train.shape)
print(X_dev.shape)
print(y_dev.shape)

In [ ]:
y_train.isnull().unique()

In [ ]:
y_dev.isnull().unique()

In [ ]:
X_train.info(show_counts=True)

In [ ]:
X_train.mean()

In [ ]:
X_dev.mean()

In [ ]:
X_dev.info(show_counts=True)

In [ ]:
X_dev

In [ ]:
def normalize(x):
    for cols in x.columns:
        mean=x[cols].mean()
        std=x[cols].std()
        if std=0:
            x[cols]=x[cols]*0
        else:
            x[cols]=(x[cols]-mean)/std
    return x
X_train=normalize(X_train)
X_dev=normalize(X_dev)

In [ ]:
X_train

In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# XGBoost with Grid Search
xgbb = xgb.XGBRegressor()

param_grid = {
    "booster": ['gbtree', 'dart'],
    "eta": [0.01, 0.06, 0.1, 0.7],
    "gamma": [0, 0.5, 1],
    "max_depth": [5, 8, 10, 13, 15],
    "lambda": [0, 1, 2],
    "alpha": [0, 1, 2]
}

xgb_model = GridSearchCV(estimator=xgbb, cv=5, param_grid=param_grid, verbose=2)
xgb_model.fit(X_train, y_train)

print(xgb_model.best_score_)
print(xgb_model.best_estimator_.get_params())

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)
y_pred= rf_reg.predict(X_dev)
print("Accuracy on Traing set: ",rf_reg.score(X_train,y_train))
print("Accuracy on Testing set: ",rf_reg.score(X_dev,y_dev))

In [ ]:
x.drop('id',axis=1,inplace=True)

In [ ]:
x.drop()

In [ ]:
y_pred= rf_reg.predict(x)
y_pred

In [ ]:
output=pd.DataFrame(y_pred)
output['Expected']=y_pred


In [ ]:
output.drop(0,inplace=True,axis=1)

In [ ]:
output['Id']=output.index

In [ ]:
output=output.reindex(['Id','Expected'], axis=1)
output.to_csv('output.csv', index = False)

In [ ]:
output.to_csv('output.csv', index = False)

In [ ]:
output

In [ ]:
output['Id']=output['Id']+1

In [ ]:
output.append([[0],[0]])

In [ ]:
ex=output.head(66)


In [ ]:
f=output.append(ex)
f

In [ ]:
output

In [ ]:
x['seats'].unique()

In [ ]:
x[x['seats']=='14.3 kmpl']='5.'

In [ ]:
output

In [ ]:
pd.read_csv('./output.csv')